# START


## <어느 시점에 얼만큼 어떤 것을, 어느 기간동안 >


그래서 결국에 어떤 목차가 되어야 맞는지는 잘 모르겠다! 이지만,
지금 진행할 content0 의 목차를 고민해보자면

### Content 0 : 기간 별, 지역별 상품 수요량 확인
#### 1. fc 월별 데이터 합치기(수요의 흐름을 보고싶은 것이기 때문에)
#### 2. fc 월별 데이터 중 지역별 재고 수요와 1%정도의 관련도 없어보이는 column 삭제하기
**칼럼 코드를 한국어로 잘 바꿔서 주석만 달아놓기**
#### 3. 날짜 별로 상품 수요량 확인하기 (시각화 하기)
#### 4. 지역 별로 상품 수요량 확인하기 (시각화 하기)
#### 5.

### Content1 : 다른 데이터 수집하기
분명 어느 정도 데이터 분석을 했고 주제 파악을 했다면 필요한 데이터가 어떤 것인지 확인될 것임. 

In [55]:
import sys
import numpy as np
import pandas as pd

### 1. csv 파일 df로 불러오기

In [56]:
fc3_df = pd.read_csv('data_2021cj\data_ecommerceFC\FC_3.csv',encoding ='utf-8')

C:\anan\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [57]:
fc3_df.head(3)

,CORP_ID,REF_ORD_NO,BKG_NO,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_SEQ,ITEM_CD,...,DLVSUBCLSFCD,INS_ID,INS_DATE,INS_TIME,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2
0,KX007,2021030365922551,KX007210303370934,7,20210303,35510,90001302,22400.0,2,134265392,...,1,iNEXBatch,20210303,131425,3.0,N,세종특별자치시,NaN,서울특별시,서대문구
1,KX007,2021030365922951,KX007210303371080,7,20210303,35521,90001441,20900.0,1,4610261043_2,...,4,iNEXBatch,20210303,131425,50.0,N,경기도,광주시,경상남도,김해시
2,KX007,2021030365924361,KX007210303371081,7,20210303,35513,90001441,37900.0,1,4936599538_1,...,0,iNEXBatch,20210303,131425,53.0,N,경기도,광주시,경상남도,거제시


In [58]:
fc3_df.shape
#32개의 요소에 대한 309352-1 개의 정보

(309352, 32)

아래의 정보에 대해 조금의 설명을 달아보자면, Dtype 의 object는 문자열이라 생각하면 되고, 지금 SHPR_ADDR_2를 제외한 모든 COLUMN들이 30만개 이상의 NON-NULL임을 확인할 수 있다.


In [59]:
fc3_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309352 entries, 0 to 309351
Data columns (total 32 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   CORP_ID             309352 non-null  object 
 1   REF_ORD_NO          309352 non-null  object 
 2   BKG_NO              309349 non-null  object 
 3   BKG_TYP             309352 non-null  int64  
 4   BKG_DATE            309352 non-null  int64  
 5   BKG_TIME            309352 non-null  int64  
 6   SHPR_CD             309352 non-null  int64  
 7   INV_AMT             305414 non-null  float64
 8   ITEM_SEQ            309352 non-null  int64  
 9   ITEM_CD             309352 non-null  object 
 10  BRAND_NM            301758 non-null  object 
 11  ITEM_QTY            309352 non-null  int64  
 12  ITEM_AMT            306284 non-null  float64
 13  IF_YN               309352 non-null  object 
 14  ORDER_CRT_DATETIME  301758 non-null  float64
 15  DLV_DV              309220 non-nul

- count : non-null인 데이터 건수
- mean : 전체 데이터의 평균값
- std : 표준편차
- min : 최솟값

In [60]:
fc3_df.describe()

,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_SEQ,ITEM_QTY,ITEM_AMT,ORDER_CRT_DATETIME,DLV_DV,INS_DATE,INS_TIME,POST_ZONE
count,309352.00000,3.093520e+05,309352.000000,3.093520e+05,3.054140e+05,309352.000000,309352.000000,3.062840e+05,3.017580e+05,309220.0,3.093520e+05,309352.000000,309220.000000
mean,7.00043,2.021032e+07,95035.831852,9.000141e+07,5.675440e+04,1.403576,2.030234,4.000405e+04,2.021032e+13,1.0,2.021032e+07,145352.291160,25.587055
std,0.02073,9.171140e+00,61630.865074,1.016592e+02,6.375390e+04,1.015545,294.231886,5.156776e+04,9.513625e+06,0.0,1.187998e+01,60230.814456,18.232241
min,7.00000,2.021030e+07,0.000000,9.000130e+07,0.000000e+00,1.000000,1.000000,0.000000e+00,2.021022e+13,1.0,2.021023e+07,4.000000,1.000000
25%,7.00000,2.021031e+07,42145.000000,9.000130e+07,1.990000e+04,1.000000,1.000000,1.335000e+04,2.021031e+13,1.0,2.021031e+07,104242.000000,10.000000
50%,7.00000,2.021032e+07,100004.000000,9.000134e+07,3.780000e+04,1.000000,1.000000,2.090000e+04,2.021032e+13,1.0,2.021032e+07,144244.000000,21.000000
75%,7.00000,2.021032e+07,133351.000000,9.000154e+07,7.160000e+04,1.000000,1.000000,4.380000e+04,2.021032e+13,1.0,2.021032e+07,201428.000000,42.000000
max,8.00000,2.021033e+07,235959.000000,9.000158e+07,4.760000e+06,35.000000,163200.000000,4.760000e+06,2.021033e+13,1.0,2.021041e+07,235637.000000,99.000000


### 2. 여러 fc 데이터 파일 하나로 합치기

In [61]:
fc4_df = pd.read_csv('data_2021cj\data_ecommerceFC\FC_4.csv')
fc5_df = pd.read_csv('data_2021cj\data_ecommerceFC\FC_5.csv')
fc6_df = pd.read_csv('data_2021cj\data_ecommerceFC\FC_6.csv')


C:\anan\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\anan\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [62]:
allFC_df = pd.concat([fc3_df,fc4_df, fc5_df, fc6_df])

In [63]:
allFC_df.head(10)

,CORP_ID,REF_ORD_NO,BKG_NO,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_SEQ,ITEM_CD,...,DLVSUBCLSFCD,INS_ID,INS_DATE,INS_TIME,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2
0,KX007,2021030365922551,KX007210303370934,7,20210303,35510,90001302,22400.0,2,134265392,...,1,iNEXBatch,20210303,131425,3.0,N,세종특별자치시,NaN,서울특별시,서대문구
1,KX007,2021030365922951,KX007210303371080,7,20210303,35521,90001441,20900.0,1,4610261043_2,...,4,iNEXBatch,20210303,131425,50.0,N,경기도,광주시,경상남도,김해시
2,KX007,2021030365924361,KX007210303371081,7,20210303,35513,90001441,37900.0,1,4936599538_1,...,0,iNEXBatch,20210303,131425,53.0,N,경기도,광주시,경상남도,거제시
3,KX007,2021030365928921,KX007210303371205,7,20210303,35608,90001542,69800.0,1,3858391637_1,...,1,iNEXBatch,20210303,131425,6.0,N,경기도,광주시,서울특별시,서초구
4,KX007,2021030365930451,KX007210303370967,7,20210303,35551,90001341,73700.0,1,4631307460,...,1,iNEXBatch,20210303,131425,10.0,N,경기도,광주시,경기도,김포시
5,KX007,2021030365930451,KX007210303370967,7,20210303,35551,90001341,73700.0,2,2475138283,...,1,iNEXBatch,20210303,131425,10.0,N,경기도,광주시,경기도,김포시
6,KX007,2021030365930451,KX007210303370967,7,20210303,35551,90001341,73700.0,3,4571859898-1,...,1,iNEXBatch,20210303,131425,10.0,N,경기도,광주시,경기도,김포시
7,KX007,2021030365933981,KX007210303370935,7,20210303,35538,90001302,14900.0,1,603215395_1,...,1p,iNEXBatch,20210303,131425,47.0,N,세종특별자치시,NaN,부산광역시,연제구
8,KX007,2021030365934611,KX007210303370936,7,20210303,35732,90001302,19800.0,1,273205138_4,...,1,iNEXBatch,20210303,131425,50.0,N,경기도,광주시,경상남도,김해시
9,KX007,2021030365939621,KX007210303370968,7,20210303,35557,90001341,37900.0,1,5161458796,...,2,iNEXBatch,20210303,131426,4.0,N,경기도,광주시,서울특별시,성동구


In [64]:
allFC_df.shape

(1786374, 32)

In [65]:
allFC_df.describe()

,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_SEQ,ITEM_QTY,ITEM_AMT,ORDER_CRT_DATETIME,DLV_DV,INS_DATE,INS_TIME,POST_ZONE
count,1.786374e+06,1.786374e+06,1.786374e+06,1.786374e+06,1.758363e+06,1.786374e+06,1.786374e+06,1.767644e+06,1.744413e+06,1785331.0,1.786374e+06,1.786374e+06,1.785331e+06
mean,7.000586e+00,2.021050e+07,9.722577e+04,9.000148e+07,7.351862e+04,1.380549e+00,1.780672e+00,5.395772e+04,2.021050e+13,1.0,2.021050e+07,1.422477e+05,2.578878e+01
std,2.420249e-02,1.113475e+02,6.375165e+04,1.423993e+02,1.051609e+05,1.294329e+00,1.306268e+02,8.097853e+04,1.113782e+08,0.0,1.113587e+02,6.171412e+04,1.830104e+01
min,7.000000e+00,2.021030e+07,0.000000e+00,9.000130e+07,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,2.021022e+13,1.0,2.021023e+07,4.000000e+00,1.000000e+00
25%,7.000000e+00,2.021042e+07,4.203000e+04,9.000130e+07,2.180000e+04,1.000000e+00,1.000000e+00,1.580000e+04,2.021042e+13,1.0,2.021042e+07,1.035280e+05,1.000000e+01
50%,7.000000e+00,2.021052e+07,1.000565e+05,9.000152e+07,3.990000e+04,1.000000e+00,1.000000e+00,2.480000e+04,2.021052e+13,1.0,2.021052e+07,1.428410e+05,2.100000e+01
75%,7.000000e+00,2.021061e+07,1.358140e+05,9.000158e+07,8.680000e+04,1.000000e+00,1.000000e+00,5.960000e+04,2.021061e+13,1.0,2.021061e+07,1.949250e+05,4.200000e+01
max,8.000000e+00,2.021063e+07,2.359590e+05,9.000184e+07,7.968000e+06,1.160000e+02,1.632000e+05,7.968000e+06,2.021063e+13,1.0,2.021073e+07,2.356380e+05,9.900000e+01


In [66]:
allFC_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1786374 entries, 0 to 676911
Data columns (total 32 columns):
 #   Column              Dtype  
---  ------              -----  
 0   CORP_ID             object 
 1   REF_ORD_NO          object 
 2   BKG_NO              object 
 3   BKG_TYP             int64  
 4   BKG_DATE            int64  
 5   BKG_TIME            int64  
 6   SHPR_CD             int64  
 7   INV_AMT             float64
 8   ITEM_SEQ            int64  
 9   ITEM_CD             object 
 10  BRAND_NM            object 
 11  ITEM_QTY            int64  
 12  ITEM_AMT            float64
 13  IF_YN               object 
 14  ORDER_CRT_DATETIME  float64
 15  DLV_DV              float64
 16  REF_ITEM_SEQ        object 
 17  ORDER_IDX           object 
 18  ORDER_YN            object 
 19  DLVPREARRBRANCD     object 
 20  DLVPREARREMPNICKNM  object 
 21  DLVCLSFCD           object 
 22  DLVSUBCLSFCD        object 
 23  INS_ID              object 
 24  INS_DATE            int64

In [67]:
#concatenate 한 df 를 csv 파일로 만들기
allFC_df.to_csv("allFC_df.csv")

### 3. 다음은 굳이 코드로 표시되어 있는 정보들을 처리하기 쉽게 만들어본다.   
3-1) CORP_ID 가 곤지암 : KX007, 군포 : GP001 이런식으로 되어 있는데 곤지암을 0, 군포를 1로 바꾼다.

In [68]:
allFC_df.loc[(allFC_df['CORP_ID']=='KX007'),'CORP_ID']='0'
allFC_df.loc[(allFC_df['CORP_ID']=='GP001'),'CORP_ID']='1'

In [69]:
allFC_df.head(5)

,CORP_ID,REF_ORD_NO,BKG_NO,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_SEQ,ITEM_CD,...,DLVSUBCLSFCD,INS_ID,INS_DATE,INS_TIME,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2
0,0,2021030365922551,KX007210303370934,7,20210303,35510,90001302,22400.0,2,134265392,...,1,iNEXBatch,20210303,131425,3.0,N,세종특별자치시,NaN,서울특별시,서대문구
1,0,2021030365922951,KX007210303371080,7,20210303,35521,90001441,20900.0,1,4610261043_2,...,4,iNEXBatch,20210303,131425,50.0,N,경기도,광주시,경상남도,김해시
2,0,2021030365924361,KX007210303371081,7,20210303,35513,90001441,37900.0,1,4936599538_1,...,0,iNEXBatch,20210303,131425,53.0,N,경기도,광주시,경상남도,거제시
3,0,2021030365928921,KX007210303371205,7,20210303,35608,90001542,69800.0,1,3858391637_1,...,1,iNEXBatch,20210303,131425,6.0,N,경기도,광주시,서울특별시,서초구
4,0,2021030365930451,KX007210303370967,7,20210303,35551,90001341,73700.0,1,4631307460,...,1,iNEXBatch,20210303,131425,10.0,N,경기도,광주시,경기도,김포시


크힉 순간 저 조건에 맞는 행들의 모든 열 값을 0으로 바꾸게 해버려 다 날라가는 상태가 발생했었다 ,, ㅎ 그래서 처음부터 다시 돌렸다,,

3-2) REF_ORD_NO 는 고객 주문번호, 해당 테이블 key 값, 네이버 채번,, 인데 음 BKG_DATE(주문 날짜) column에 들어가있는 값 + ,,65922551같은 숫자들은 모르겠는데,, 일단 굳이 앞의 주문날짜를 살려 놓을 필요는 없으니깐 삭제해버리고 뒷 부분의 숫자들은 어떤 값들인진 모르겠지만 해석을 못해먹겠고 코드,,짜기 귀찮으니 삭제해버린다.

In [70]:
allFC_df.drop(['REF_ORD_NO'],axis=1, inplace=True)#원본 데이터에서 날린거임!

3-3) BKG_NO는 대한통운 주문번호 (예약번호)이고 솔직히 다 필요없다고 생각해서 아예 column 전체를 날렸다.

In [71]:
allFC_df.drop(['BKG_NO'],axis=1, inplace=True) 

In [72]:
allFC_df.head(3)

,CORP_ID,BKG_TYP,BKG_DATE,BKG_TIME,SHPR_CD,INV_AMT,ITEM_SEQ,ITEM_CD,BRAND_NM,ITEM_QTY,...,DLVSUBCLSFCD,INS_ID,INS_DATE,INS_TIME,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2
0,0,7,20210303,35510,90001302,22400.0,2,134265392,단일상품,1,...,1,iNEXBatch,20210303,131425,3.0,N,세종특별자치시,NaN,서울특별시,서대문구
1,0,7,20210303,35521,90001441,20900.0,1,4610261043_2,조합형옵션상품,1,...,4,iNEXBatch,20210303,131425,50.0,N,경기도,광주시,경상남도,김해시
2,0,7,20210303,35513,90001441,37900.0,1,4936599538_1,조합형옵션상품,1,...,0,iNEXBatch,20210303,131425,53.0,N,경기도,광주시,경상남도,거제시


3-4) 아무래도 날려야할게 너무 많이 보여서 한 번에 날리기로 결심했다,,
- SHPR_CD : 고객사 코드
- ITEM_CD : 품목순번
- BRAND_NM : 품목명 (판매자가 설정해놓는 것이라고 한다. 품목의 종류를 확인하기 어렵다...)

In [76]:
allFC_df.drop(['SHPR_CD','ITEM_CD','BRAND_NM'],axis=1, inplace=True)

In [77]:
allFC_df.head(3)

,CORP_ID,BKG_TYP,BKG_DATE,BKG_TIME,INV_AMT,ITEM_SEQ,ITEM_QTY,ITEM_AMT,IF_YN,ORDER_CRT_DATETIME,...,DLVSUBCLSFCD,INS_ID,INS_DATE,INS_TIME,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2
0,0,7,20210303,35510,22400.0,2,1,14900.0,N,2.021030e+13,...,1,iNEXBatch,20210303,131425,3.0,N,세종특별자치시,NaN,서울특별시,서대문구
1,0,7,20210303,35521,20900.0,1,1,20900.0,N,2.021030e+13,...,4,iNEXBatch,20210303,131425,50.0,N,경기도,광주시,경상남도,김해시
2,0,7,20210303,35513,37900.0,1,1,37900.0,N,2.021030e+13,...,0,iNEXBatch,20210303,131425,53.0,N,경기도,광주시,경상남도,거제시


3-5) 계속 날리는데 사실 이 화면에 모든 column 항목들이 안떠서 계속 삭제하고 보고 ,, 하는 중이다. ㅋㅋㅋ
- IF_YN : 수신 여부를 확인하는 것인데 N이면 수신이 되지 않았다는 거임,,,? 어쨌든 일단은 날리고.. (뭔가 불안하니깐 inplace=False로 맞추겠음.)
- ORDER_CRT_DATETIME : 주문생성시간인데 이거 딱봐도 앞의 주문 날짜 + 주문시간한 거 합친 데이터임,, 왜 넣어놨냐고 용량아깝게;
.. 뒤에부터도 같이 삭제해야지.. 마지막 4열들은 위치 값들이니 살려놓고 
- SPLIT_EXEC_YN : 배송처별 주문분할 여부,,,,, 이거는 친구들과 함께 이야기해보구
- POST_ZONE : 권역구분 : 이것도 마찬가지,, 근데 숫자로 표시되어있네 신기하네
- INS 시리즈는 다 날린다,, 전부 입력자ID, 입력일자, 입력시간이고 아직 의미있는지 모르겠음..   
일단 이렇게만!

In [78]:
allFC_df1 = allFC_df.drop(['IF_YN','ORDER_CRT_DATETIME','INS_ID','INS_DATE','INS_TIME'],axis =1, inplace = False) 

In [79]:
allFC_df1.head(5)

,CORP_ID,BKG_TYP,BKG_DATE,BKG_TIME,INV_AMT,ITEM_SEQ,ITEM_QTY,ITEM_AMT,DLV_DV,REF_ITEM_SEQ,...,DLVPREARRBRANCD,DLVPREARREMPNICKNM,DLVCLSFCD,DLVSUBCLSFCD,POST_ZONE,SPLIT_EXEC_YN,SHPR_ADDR_1,SHPR_ADDR_2,CNEE_ADDR_1,CNEE_ADDR_2
0,0,7,20210303,35510,22400.0,2,1,14900.0,1.0,2021030323885041,...,7484,C06,5Z73,1,3.0,N,세종특별자치시,NaN,서울특별시,서대문구
1,0,7,20210303,35521,20900.0,1,1,20900.0,1.0,2021030323885721,...,6427,M09,6R67,4,50.0,N,경기도,광주시,경상남도,김해시
2,0,7,20210303,35513,37900.0,1,1,37900.0,1.0,2021030323887981,...,6158,L32,9Y25,0,53.0,N,경기도,광주시,경상남도,거제시
3,0,7,20210303,35608,69800.0,1,1,69800.0,1.0,2021030323895281,...,8909,D21,1N65,1,6.0,N,경기도,광주시,서울특별시,서초구
4,0,7,20210303,35551,73700.0,1,1,39800.0,1.0,2021030323897651,...,2777,R15,3D84,1,10.0,N,경기도,광주시,경기도,김포시


아씨 칼럼 지워도 지워도 계속 많다.. 내일 할래,,,,,,